In [1]:
import torch
from transformers import TFLongformerModel, LongformerTokenizer
import pandas as pd
from sklearn import model_selection
import tensorflow as tf
import tensorflow_text as text

In [2]:
# Load in dataset
data = pd.read_csv("../../data/sensitivity_data/sensitivity_dataset.csv")
data = data[["Filename","Date","Sensitivity","Document"]]

# Train / Test split
train_x, test_x, train_y, test_y = model_selection.train_test_split(data['Document'],data['Sensitivity'],test_size=0.2,random_state=5)

In [3]:
model = TFLongformerModel.from_pretrained('allenai/longformer-base-4096')
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

input_ids = tf.keras.Input(shape=(4096),dtype='int32')
attention_mask = tf.keras.Input(shape=(4096), dtype='int32')
global_attention_mask = None
outputs = model(input_ids, attention_mask=attention_mask,global_attention_mask=global_attention_mask)
sequence_output = outputs.last_hidden_state
pooled_output = outputs.pooler_output
#reshaped = tf.reshape(pooled_output,[-1, 768, 1])
#lstm = tf.keras.layers.LSTM(32,return_sequences=True)(reshaped)
dense = tf.keras.layers.Dense(1,activation='sigmoid')(pooled_output)
longformer_model = tf.keras.Model(inputs = [input_ids, attention_mask],outputs = dense)
longformer_model.summary()

Some layers from the model checkpoint at allenai/longformer-base-4096 were not used when initializing TFLongformerModel: ['lm_head']
- This IS expected if you are initializing TFLongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFLongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFLongformerModel were initialized from the model checkpoint at allenai/longformer-base-4096.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFLongformerModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4096)

In [4]:
optimiser = tf.keras.optimizers.Adam()
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
metrics = tf.keras.metrics.BinaryAccuracy()

longformer_model.compile(optimizer=optimiser,loss='binary_crossentropy',metrics=metrics)

In [5]:
#model = TFLongformerModel.from_pretrained('allenai/longformer-base-4096')
#tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
inputs = tokenizer(list(train_x), return_tensors="tf",padding=True,truncation=True)
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

In [7]:
history = longformer_model.fit(x=[input_ids,attention_mask],y=train_y,epochs=5,batch_size=1,verbose=2)

Epoch 1/5
